In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import random
random.seed(5)
import sys
import random
sys.path.append('../src')
from models import word_set_to_mtx, save_pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

we_model = Word2Vec.load(we_model_dir+'/model.gensim')
print ('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')


'''
# Caliskan GloVe
glove_file = '../data/external/glove.6B/glove.6B.50d.txt'
_ = glove2word2vec(glove_file, '../data/interim/tmp.txt')
we_model = KeyedVectors.load_word2vec_format('../data/interim/tmp.txt')
print('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')
'''

loading done!


C:\Users\danie\Miniconda3\envs\semproject\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


Total words: 400000


In [11]:
THRESHOLD_BIASES_PATH_2NDORDER = '../data/processed/threshold_biases_2ndorder.pickle'

In [12]:
vocabulary = list(we_model.wv.vocab.keys())
random.choices(vocabulary, k=3)

['wftv', 'stollwerck', 'dansville']

In [13]:
set_1 = random.choices(vocabulary, k=3)
set_2 = random.choices(vocabulary, k=3)

In [14]:
set_1 = np.array([[1, 1, 1], [2,5,10], [3,5,10]])
set_2 = np.array([[1, 1, 1], [5, 5, 5], [10,10,10]])
zip(set_1, set_2)
[print(u,v) for u,v in zip(set_1, set_2)]

[1 1 1] [1 1 1]
[ 2  5 10] [5 5 5]
[ 3  5 10] [10 10 10]


[None, None, None]

In [19]:
def matrix_cosine(x, y):
    '''If x is NxM array and y is NxM array, this function
    calculates row-wise cosine similary, so the results is a
    array with shape (N,).'''
    return np.einsum('ij,ij->i', x, y) / (
              np.linalg.norm(x, axis=1) * np.linalg.norm(y, axis=1)
    )
print(type(set_1))
print(matrix_cosine(set_1, set_2).shape)
matrix_cosine(set_1, set_2)

<class 'numpy.ndarray'>
(3,)


array([1.        , 0.86415857, 0.8977584 ])

In [17]:
def get_biases(we_model, k_pairs=500000, order='second'):
    #TODO: refactor: remove 'order' argument, no longer necessary
    vocabulary = list(we_model.wv.vocab.keys())
    set_1 = random.choices(vocabulary, k=k_pairs)
    set_2 = random.choices(vocabulary, k=k_pairs)
    arr_1 = word_set_to_mtx(we_model, set_1)
    arr_2 = word_set_to_mtx(we_model, set_2)
    biases = matrix_cosine(arr_1, arr_2)
    save_pickle(biases, THRESHOLD_BIASES_PATH_2NDORDER)
    return biases
get_biases(we_model)

array([0.14972538, 0.03376701, 0.23382741, ..., 0.098508  , 0.08962049,
       0.02198978], dtype=float32)